In [1]:
from src.payment import MeansOfPaymentType

mop1 = MeansOfPaymentType('H_Cash')
mop2 = MeansOfPaymentType('F_CBDC')
mop3 = MeansOfPaymentType('Deposit')

mops = set([mop1, mop2, mop3])

In [2]:
print(mop1)

H_Cash


In [23]:
from dataclasses import dataclass, field


class Person:
    def __init__(self, name, age = 10) -> None:
        self.name = name
        self.age = age


@dataclass(order=True)
class PersonDC:
    name:str = field(compare=False)
    age: int = 10

In [24]:
p1 = PersonDC('a', age=30)
p2 = PersonDC('b', age=40)

In [26]:
print(p1 < p2)

True
